In [1]:
from IPython.core.display import display, HTML
from scipy.spatial.transform import Rotation
display(HTML("<style>.container { width:100% !important; }</style>"))
import h5py
from matplotlib import pyplot as plt
import torch
import numpy as np
from pytorch3dunet.unet3d.losses import *
from pytorch3dunet.unet3d.metrics import MeanIoU
from pathlib import Path
import glob
import os
import h5py
import numpy as np
import prody
import prody as pr
from potsim2 import PotGrid
import sklearn
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import f1_score
import os
import pandas as pd


import matplotlib
#matplotlib.use('TkAgg')

%matplotlib inline

ModuleNotFoundError: No module named 'pytorch3dunet'

In [2]:
class DiceProbLoss(DiceLoss):
    def __init__(self):
        super(DiceProbLoss, self).__init__(normalization='none')

class MeanIoUBin(MeanIoU):
    def __init__(self):
        super(MeanIoUBin, self).__init__(is_binarized=True)

class BCEDiceBinLoss(nn.Module):
    """Linear combination of BCE and Dice losses"""

    def __init__(self, alpha, beta):
        super(BCEDiceBinLoss, self).__init__()
        self.alpha = alpha
        self.bce = nn.BCELoss()
        self.beta = beta
        self.dice = DiceBinLoss()

    def forward(self, input, target):
        return self.alpha * self.bce(input, target) + self.beta * self.dice(input, target)


initLosses = {
    "BCE": nn.BCELoss,
    "Dice": DiceProbLoss,
    "MeanIoU": MeanIoU
}

NameError: name 'DiceLoss' is not defined

In [25]:
basepred = Path('/home/lorenzo/3dunet-cavity/runs/run_210623_gpu/predictions')
baseorig = Path('/home/lorenzo/deep_apbs/destData/refined-set_filter')

def genDataSets():

    for predfname in glob.glob(str(basepred / '*_grids_predictions.h5')):
        name = Path(predfname).name.split('_')[0]
        labelfname = baseorig / name / f'{name}_grids.h5'
        
        if not os.path.exists(labelfname):
            print(f'{labelfname} does not exist.')
            continue
        
        labelT = torch.tensor(h5py.File(labelfname)['label'], dtype=torch.float32)
        labelT = labelT[None,None]
        predT = torch.tensor(h5py.File(predfname)['predictions'])
        predT = predT[None]
        
        yield name, (predT, labelT)

In [26]:
class RunningAverage:
    def __init__(self, loss):
        self.count = 0
        self.sum = 0
        self.loss = loss
        self.losses = {}

    def update(self, pred, label, name):
        self.count += 1
        loss = self.loss(pred, label).item()
        self.sum += loss
        self.losses[name] = loss

    def value(self):
        return self.sum / self.count
    
class AverageLosses:
    def __init__(self, losses, kwargs=None):
        if kwargs is None:
            kwargs = {name:{} for name in losses.keys()}
        self.losses = {name: RunningAverage(loss(**kwargs[name])) for name,loss in losses.items()}

    def update(self, pred, label, prot):
        for name in self.losses.keys():
            self.losses[name].update(pred,label, prot)
        
    def value(self):
        return {name: loss.value() for name,loss in self.losses.items()}

In [27]:
OracleLoss = AverageLosses(initLosses)
UnetLoss = AverageLosses(initLosses)
RandLoss = AverageLosses(initLosses)
RandUnitLoss = AverageLosses(initLosses)
ZeroLoss = AverageLosses(initLosses)
UnitLoss = AverageLosses(initLosses)

for name, (predT, labelT) in genDataSets():
    constPred = torch.zeros_like(labelT)
    ZeroLoss.update(constPred, labelT, name)

    constPred[:] = 1
    UnitLoss.update(constPred, labelT, name)

    UnetLoss.update(predT, labelT, name)

    OracleLoss.update(labelT, labelT, name)

    randPred = torch.rand(size=constPred.size(), dtype=constPred.dtype, device=constPred.device, requires_grad=False)
    RandLoss.update(randPred, labelT, name)
    
    randPred[randPred < 0.5] = 0
    randPred[randPred > 0.5] = 1
    RandUnitLoss.update(randPred, labelT, name)

/home/lorenzo/deep_apbs/destData/refined-set_filter/1oba/1oba_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2epn/2epn_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1tkb/1tkb_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1rnt/1rnt_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2h21/2h21_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2am4/2am4_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1x8d/1x8d_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/2euk/2euk_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1lnm/1lnm_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1o1s/1o1s_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-set_filter/1r9l/1r9l_grids.h5 does not exist.
/home/lorenzo/deep_apbs/destData/refined-se

In [28]:
RandUnitLoss.value(), RandLoss.value(), UnitLoss.value(), ZeroLoss.value(),OracleLoss.value()

({'BCE': 49.99950144841121,
  'Dice': 0.997966271180373,
  'MeanIoU': 0.0010180229527087738},
 {'BCE': 0.9999702343573937,
  'Dice': 0.9969496108018435,
  'MeanIoU': 0.0010180229527087738},
 {'BCE': 99.89769275371845,
  'Dice': 0.997956259892537,
  'MeanIoU': 0.001023037529263932},
 {'BCE': 0.1023037525323721, 'Dice': 1.0, 'MeanIoU': 0.0},
 {'BCE': 0.0, 'Dice': 0.0, 'MeanIoU': 1.0})

In [32]:
UnetLoss.losses['MeanIoU'].losses

{'1sw2': 0.4645071029663086,
 '1fiv': 0.6156509518623352,
 '1a4r': 0.17319780588150024,
 '4gbd': 0.6333988904953003,
 '5g1z': 0.12134632468223572,
 '5gja': 0.3489736020565033,
 '2fqt': 0.29123446345329285,
 '4r75': 0.567950963973999,
 '4non': 0.15870153903961182,
 '3fjg': 0.5126749873161316,
 '3s0e': 0.493194043636322,
 '1dy4': 0.5372849106788635,
 '1dqn': 0.2584525942802429,
 '1m48': 0.027618248015642166,
 '5yz2': 0.20882698893547058,
 '4fxp': 0.48952972888946533,
 '5mxf': 0.0,
 '1ctt': 0.0,
 '1ceb': 0.2004687339067459,
 '5km9': 0.18358531594276428,
 '2dw7': 0.008737157098948956,
 '1t7d': 0.22022944688796997,
 '5tcj': 0.03971793130040169,
 '5vij': 0.4260115623474121,
 '4jwk': 0.23512376844882965,
 '5tuz': 0.09520634263753891}

In [ ]:
thress = np.linspace(0.1,1.0,10)

kwargs = {f"MeanIoU_{thres}": {'thres':thres} for thres in thress}
initLosses = {f"MeanIoU_{thres}": MeanIoU for thres in thress}

UnetLosses = AverageLosses(initLosses, kwargs)

for name, (predT, labelT) in genDataSets():
    UnetLosses.update(predT, labelT, name)

In [ ]:
UnetLosses.losses['MeanIoU'].losses

## Scratch

In [5]:
import h5py

fname =  '/home/lorenzo/3dunet-cavity/runs/test_3/tmp/4qij/grids.h5'

h5 = h5py.File(fname, 'r')

In [9]:
h5['raws'][0][0].sum()

5655.0

In [11]:
h5['raws'][0][1].mean(), h5['raws'][0][1].std()

(0.019379736825611633, 2.07141037408601)

In [3]:
with h5py.File(h5path2, 'w') as h52:
    h52.create_dataset('raws', data=h5['raws'][0])
    h52.create_dataset('labels', data=h5['labels'][0].astype(float))

In [6]:
with h5py.File(h5path2, 'r') as h52:
    df = pd.Series(h52['labels'][:].flatten())
df.value_counts()

0.0    4164884
1.0       8397
dtype: int64

In [39]:
name = '4us3'
angles = [1.5273467402269651, 1.3922370730364215, -0.21520977985323195]

In [41]:
m = Rotation.from_euler('xyz', [1.5273467402269651, 1.3922370730364215, -0.21520977985323195])
r = m.as_matrix()

In [43]:
Rotation.from_euler()

<function Rotation.from_euler>

In [33]:
import yaml
import h5py

conf = yaml.safe_load(open(rf'/home/lorenzo/3dunet-cavity/runs/210805_pdb/run_config.yml','r'))

In [ ]:
h5py.File()

In [32]:
len(conf['val']) * (arr.size * 8) / 10**6

6977.725832

In [12]:
import numpy as np
import sys

arr = np.random.random(size=(161,161,161))

In [18]:
arr.dtype

dtype('float64')

33.386248

In [21]:
314 - 256

58

In [15]:
sys.getsizeof(arr)

33386376

In [2]:
import h5py
import numpy as np
import prody
import os
from potsim2 import PotGrid

predbase = f'/home/lorenzo/3dunet-cavity/runs/run_210628_gpu'

datadir = '/home/lorenzo/deep_apbs/destData/refined-set_filter'
name = '1a4r'

predfname = f'{predbase}/predictions/{name}_grids_predictions.h5'
pdbfname = f'{datadir}/{name}/{name}_selected.pdb'
ligandfname = f'{datadir}/{name}/{name}_ligand.pdb'

grids = h5py.File(predfname)
pred = np.array(grids['predictions'][0])
predbin = pred  > 0.5

structure = prody.parsePDB(pdbfname)
grid = PotGrid(f"{datadir}/{name}/{name}_selected.pdb", f"{datadir}/{name}/{name}_grid.dx.gz")

ligand = prody.parsePDB(ligandfname)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/home/lorenzo/3dunet-cavity/runs/run_210628_gpu/predictions/1a4r_grids_predictions.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
complx = ligand + structure
lresname = ligand.getResnames()[0]
pocket = complx.select(f'same residue as exwithin 4.5 of resname {lresname}')
prody.writePDB(f'{datadir}/{name}/{name}_pocket.pdb', pocket)
pocket

<Selection: 'same residue as... of resname GNH' from 1a4r_ligand + 1a4r_selected (246 atoms)>

In [5]:
print(min(grid.edges[0]), max(grid.edges[0]))
print(min(grid.edges[1]), max(grid.edges[1]))
print(min(grid.edges[2]), max(grid.edges[2]))

8.8395 169.8395
-53.3765 107.6235
-86.7575 74.2425


In [6]:
print(min(x[0] for x in structure.getCoords()), max(x[0] for x in structure.getCoords()))
print(min(x[1] for x in structure.getCoords()), max(x[1] for x in structure.getCoords()))
print(min(x[2] for x in structure.getCoords()), max(x[2] for x in structure.getCoords()))

60.64 118.039
6.425 47.822
-27.466 14.951


In [71]:
coords = []

for i,coord in enumerate(structure.getCoords()):
    x,y,z = coord
    binx = int((x - min(grid.edges[0])) / grid.delta[0])
    biny = int((y - min(grid.edges[1])) / grid.delta[1])
    binz = int((z - min(grid.edges[2])) / grid.delta[2])
    
    try:
        if predbin[binx,biny,binz]:
            coords.append(i)
    except IndexError as e:
        print(x,  min(grid.edges[0]), max(grid.edges[0]), grid.delta[0])
        print(binx,biny,binz)
        raise e

In [73]:
s = structure[[coords]]
lresname = s.getResnames()[0]

In [90]:
len(prody.AtomGroup())

0

In [84]:
structure.copy()

<AtomGroup: 1a4r_selected (3004 atoms)>

In [64]:
fname1 = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1/2ydw/2ydw_selected.pdb'
fname2 = '/home/lorenzo/deep_apbs/destData/refined-set_filter_1/2ydw/2ydw_grid.dx.gz'

grid2  = PotGrid(fname1, fname2)

In [58]:
idxstr = ' '.join(map(str,s.getIndices()))
s2 = structure.select(f'same residue as index {idxstr}')
s2

<Selection: 'same residue as... 2780 2781 2783' from 1a4r_selected (845 atoms)>

In [ ]:
pocket = structure.select(f'same residue as {lresname}')

In [79]:
grid.grid = predbin.astype(int)
grid.export(f'{predbase}/{name}_predicted.dx')

In [ ]:
prody.parsePDB(pdbfname)

In [64]:
structure

<AtomGroup: 1a4r_selected (3004 atoms)>

In [63]:
prody.AtomGroup(s)

<AtomGroup: Selection 'index 57 61 64 ...80 to 2781 2783' (0 atoms; no coordinates)>

In [54]:
len(structure.getCoords()), len(coords)

(3004, 519)